### 버스 노선별 정류장수, 거리, 굴곡도

In [10]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [11]:
busroute_usage = pd.read_csv(
    filepath_or_buffer="../data/gyeongn_busroute_total.csv"
    )

### 웹 크롤링

In [12]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

In [15]:
city_name = "창원시" ## "창원시" 김해시
startnodenm = "신전" ## 롯데아울렛(기점)
routeno = 1
findtime = 10000

route_usage_data = {}


if city_name == "창원시":
    region_button_selector = "48123_48127_48125_48120_48121_48129"
    
elif city_name == "진주시":
    region_button_selector = "48170"
    
elif city_name == "통영시":
    region_button_selector = "48220"
    
elif city_name == "사천시":
    region_button_selector = "48240"
    
elif city_name == "김해시":
    region_button_selector = "48250"
    
elif city_name == "밀양시":
    region_button_selector = "48270"
        
elif city_name == "거제시":
    region_button_selector = "48310"
    
elif city_name == "양산시":
    region_button_selector = "48330"
    
elif city_name == "의령군":
    region_button_selector = "48720"
    
elif city_name == "함안군":
    region_button_selector = "48730"
    
elif city_name == "창녕군":
    region_button_selector = "48740"
    
elif city_name == "고성군":
    region_button_selector = "48820"
    
elif city_name == "남해군":
    region_button_selector = "48840"
    
elif city_name == "하동군":
    region_button_selector = "48850"
    
elif city_name == "산청군":
    region_button_selector = "48860"
    
elif city_name == "함양군":
    region_button_selector = "48870"

elif city_name == "거창군":
    region_button_selector = "48880"
    
elif city_name == "합천군":
    region_button_selector = "48890"
    
else:
    raise ValueError(f"Unknown city_name: {city_name}")


options = Options()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
driver = wb.Chrome(options = options)


## 교통카드빅데이터 종합정보시스템 -> 노선별 굴곡도 조회 url
driver.get("https://stcis.go.kr/pivotIndi/wpsPivotIndicator.do?siteGb=P&indiClss=IC05&indiSel=IC0507")

## 클릭 함수
def driver_clickable_processor(css_selector_value):
    click = WebDriverWait(driver, findtime).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_value))
    )    
    driver.execute_script("arguments[0].scrollIntoView();", click)
    driver.execute_script("arguments[0].click();", click)    
    
    return click

## 텍스트 추출 함수 
def driver_extract_text_processor(css_selector_value):
    element = WebDriverWait(driver, findtime).until(
                    EC.presence_of_element_located((
                        By.CSS_SELECTOR, 
                        css_selector_value
                        ))
                )
    return element.text

## 굴곡도 메뉴 클릭
search_menu = WebDriverWait(driver, findtime).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#ulListIndiSel > li:nth-child(4)"))
)
driver.execute_script("arguments[0].scrollIntoView();", search_menu)
driver.execute_script("arguments[0].click();", search_menu)


## 시도 박스 클릭
dropdown = Select(driver.find_element(By.XPATH, '//*[@id="searchPopZoneSd"]'))
dropdown.select_by_index(14)


## 지역 버튼 클릭 > 지역마다 클릭 버튼이 다름
dropdown = Select(driver.find_element(By.XPATH, '//*[@id="searchPopZoneSgg"]'))
dropdown.select_by_value(region_button_selector)


## 노선명 입력칸 클릭 -> 노선번호(명) 입력
route_name_input = driver_clickable_processor("#popupSearchRouteNo")
route_name_input.send_keys(f"{routeno}")


## 검색 버튼 클릭
search_button = driver_clickable_processor("#route_space1 > li.box_flex > button")


## 노선명과 기종점이 같은지 확인
routeno_len = int(WebDriverWait(driver, findtime).until(
    EC.element_to_be_clickable((By.CLASS_NAME, 'pop_box'))
).text.split()[2])

